# Programming C using SWIG

In this notebook we will show how to leverage [SWIG](http://www.swig.org/) to develop normal
C program in Jupyter environment. This is a unique feature added by the 
`pynq` package.

In [1]:
from pynq.lib import swig

## Example 1: Simple functions
Let's first look at a very simple example. With the magic `swig`,
we can specify the desired Python module name `example1`. Under the hood:
1. The interface file (`example1.i`) is generated.
2. The header (`example1.h`) is generated with the 
desired function prototypes and necessary headers.
3. The C program is compiled into a shared object file (`_example1.so`).

In [2]:
%%swig example1

int multiply(int a, int b) {
    return a*b;
}

double multiply_half(double c){
    return c*0.5;
}

Then we can easily import the module and call the C functions as if they
are available as Python functions.

In [3]:
import example1
example1.multiply(2,3)

6

In [4]:
example1.multiply_half(0.5)

0.25

**Note:** you can change the C program and recompile, but for Jupyter notebook
to reload the new context, you will need to restart the kernel
for changes in C to take effect.

## Example 2: Leverage system functions/headers
One good thing about this feature, is that we can leverage existing 
C libraries in the system. In the following example, we are leveraging the 
functions and data structures defined in the `time.h` header.

The following cell may take a while to compile, since internally the 
C program as well as its headers is being parsed.

In [5]:
%%swig example2
#include <stdio.h>
#include <time.h>

int hello() {
    time_t t = time(NULL);
    struct tm tm = *localtime(&t);
    printf("Hello from C Program!\n");
    printf("Today's date: %02d/%02d, %d\n", 
        tm.tm_mon + 1, tm.tm_mday, tm.tm_year + 1900);
    printf("Current time: %02d:%02d:%02d\n",
        tm.tm_hour, tm.tm_min, tm.tm_sec);
}

We leverage the [existing Python package `wurlitzer`](https://github.com/minrk/wurlitzer) so we can capture the 
C outputs into the Jupyter notebook. You should be able to see the printout
from the C program. This can also be used for debugging purpose.

In [6]:
from wurlitzer import sys_pipes
from example2 import hello

with sys_pipes():
    hello()

Hello from C Program!
Today's date: 04/23, 2020
Current time: 22:43:52


## Example 3: Measuring the performance

SWIG does not have optimal performance as a Python-C binding framework. 
But to get an idea of how the performance looks like, we can experiment with
the quick-sort algorithm. Below is the Python implementation.

In [7]:
def quickSortPython(arr, left, right):
    mid = int((right+left)/2)
    pivot = arr[mid]
    i, j = left, right
    while (i<=j):
        while arr[i]<pivot:
            i+=1
        while arr[j]>pivot:
            j-=1
        if i<=j:
            arr[i], arr[j] = arr[j], arr[i]
            i, j = i+1, j-1
    if left<j:
        arr = quickSortPython(arr, left, j)
    if i<right:
        arr = quickSortPython(arr, i, right)
    return arr

Next let's write a similar program in C.

In [8]:
%%swig example3

void quickSortC(int* arr, int left, int right) {
    int i = left;
    int j = right;
    int tmp;
    int pivot = arr[(left+right)/2];
    while (i <= j) {
        while (arr[i] < pivot)
           i++;
        while (arr[j] > pivot)
           j--;
        if (i <= j) {
            tmp = arr[i];
            arr[i] = arr[j];
            arr[j] = tmp;
            i++;
            j--;
        }
    }
    if (left < j)
       quickSortC(arr, left, j);
    if (i < right)
       quickSortC(arr, i, right);
}

Since we have used C arrays, we have to populate input and extract output
using the existing member functions from SWIG. 
In the following cell, we wrap the `quickSortC()` function into a 
higher-level function `quickSortSwig()`, so the API looks exactly the same 
as the `quickSortPython()`.

Note that in the following cell, `new_intArray()` and `delete_intArray()`
are used to allocate and free the memory.

In [9]:
from example3 import new_intArray, delete_intArray
from example3 import intArray_setitem, intArray_getitem
from example3 import quickSortC

def quickSortSwig(arr, left, right):
    temp = new_intArray(len(arr))
    for i in range(len(arr)):
        intArray_setitem(temp,i,int(arr[i]))

    quickSortC(temp, 0, len(arr)-1)

    result = arr
    for i in range(len(arr)):
        result[i]=intArray_getitem(temp,i)
    delete_intArray(temp)
    return result

Let's have a bit fun by testing them.

In [10]:
from copy import deepcopy
import numpy as np

size = 100
test_array0 = np.random.randint(low=1, high=100, size=size)
test_array1 = deepcopy(test_array0)
test_array2 = deepcopy(test_array0)
print("Unsorted array:")
print(test_array0)

Unsorted array:
[46 93 99 89 71 90 94 77 93 80 19 63 51 57 62 34 56 45 92 71 77 78 46 60 58
 54 45 62 33 91 43 31 89 27 78 88 99 22  3 63 53 30 22 90 66 27 59 86 53 16
 23 78 65 72 19 57 11 89 47 22 11 43  6 96 18 12 40 39  3 88  7 78 73 23 70
 39 91 67 40 31  6 49 57 52 15 31 94  2 47 99 75 99 81 89 87 44 33 84 61 81]


In [11]:
test_array1 = quickSortPython(test_array1, 0, len(test_array1)-1)
print("Sorted by Python:")
print(test_array1)

Sorted by Python:
[ 2  3  3  6  6  7 11 11 12 15 16 18 19 19 22 22 22 23 23 27 27 30 31 31 31
 33 33 34 39 39 40 40 43 43 44 45 45 46 46 47 47 49 51 52 53 53 54 56 57 57
 57 58 59 60 61 62 62 63 63 65 66 67 70 71 71 72 73 75 77 77 78 78 78 78 80
 81 81 84 86 87 88 88 89 89 89 89 90 90 91 91 92 93 93 94 94 96 99 99 99 99]


In [12]:
test_array2 = quickSortSwig(test_array2, 0, len(test_array2)-1)
print("Sorted by SWIG:")
print(test_array2)

Sorted by SWIG:
[ 2  3  3  6  6  7 11 11 12 15 16 18 19 19 22 22 22 23 23 27 27 30 31 31 31
 33 33 34 39 39 40 40 43 43 44 45 45 46 46 47 47 49 51 52 53 53 54 56 57 57
 57 58 59 60 61 62 62 63 63 65 66 67 70 71 71 72 73 75 77 77 78 78 78 78 80
 81 81 84 86 87 88 88 89 89 89 89 90 90 91 91 92 93 93 94 94 96 99 99 99 99]


You should notice the results match. Let's also time the executions of the
2 implementations (`quickSortPython` and `quickSortSwig`).

In [13]:
%%timeit

quickSortPython(np.random.randint(low=1, high=100, size=size), 0, size-1)

1000 loops, best of 3: 1.96 ms per loop


In [14]:
%%timeit

quickSortSwig(np.random.randint(low=1, high=100, size=size), 0, size-1)

1000 loops, best of 3: 662 µs per loop


Copyright (C) 2020 Xilinx, Inc